# Choosing a location for a well

For Production Company X, there is need to determine where to drill the new well.

As **data**, oil samples were provided in three regions: in each of 10,000 fields, where the quality of oil and the volume of its reserves were measured. **Project goal**: to build a machine learning model that will help determine the region where oil extraction will bring the greatest profit.

**Note:**
- When exploring a region, 500 points are explored, from which the best 200 are selected for development using machine learning;
- The budget for the development of wells in the region is 10 billion rubles.
- At current prices, one barrel of raw materials brings in 450 rubles of income. The income from each unit of the product is 450 thousand rubles, since the volume is indicated in thousands of barrels.
- After assessing the risks, it is necessary to leave only those regions in which the probability of losses is less than 2.5%. Among them, the region with the highest average profit is chosen.
- The data is synthetic: the details of the contracts and the characteristics of the deposits were not disclosed.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-and-preparing-data" data-toc-modified-id="Loading-and-preparing-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading and preparing data</a></span></li><li><span><a href="#Model-training" data-toc-modified-id="Model-training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model training</a></span></li><li><span><a href="#Preparation-for-calculating-profit" data-toc-modified-id="Preparation-for-calculating-profit-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preparation for calculating profit</a></span></li><li><span><a href="#Calculation-of-profit-and-risks" data-toc-modified-id="Calculation-of-profit-and-risks-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Calculation of profit and risks</a></span></li><li><span><a href="#Risks-and-revenue-for-each-region:" data-toc-modified-id="Risks-and-revenue-for-each-region:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Risks and revenue for each region:</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></div>

## Loading and preparing data

In [4]:
# libraries
import pandas as pd
import numpy as np
import scipy.stats as st
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from numpy.random import RandomState

# mute notifications
import warnings
warnings.filterwarnings('ignore')

# import datasets
df1 = pd.read_csv('geo_data_0.csv')
df2 = pd.read_csv('geo_data_1.csv')
df3 = pd.read_csv('geo_data_2.csv')

In [5]:
df1.head(5)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [6]:
df2.head(5)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
df3.head(5)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


**Checking basic information about dataset**

In [8]:
# function for datasets analysis

def df_info(data):
    print(data.info())
    print(data.describe())
    print('Missing values:', data.isna().sum().sum())
    print('Dubleciates:', data.duplicated().sum())

**Well 1:**

In [9]:
df_info(df1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.3437

**Well 2:**

In [10]:
df_info(df2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%         1.153055      -4.813172       2.011479      57.085625
75%         8.621015      -1.332816       3.999904     107.813044
max        29.421755      18.7340

**Well 3:**

In [11]:
df_info(df3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.174820       0.130359      59.450441
50%         0.009424      -0.009482       2.484236      94.925613
75%         1.158535       1.163678       4.858794     130.595027
max         7.238262       7.8448

Initial analysis of the provided datasets showed the following:

- each of the three datasets has 100 thousand observations;
- variables with features (f0, f1 and f2) are of the 'float64' type;
- no missing values or duplicates were found in any of the datasets.

## Model training

Based on the fact that it is necessary to predict the numerical values, linear regression will be used.

Let's write a function that will allow to perform the following data manipulations:

- split into training and validation sets in a ratio of 75:25;
- train the model and make predictions on the validation set;
- save predictions and correct answers on the validation set;
- print on the screen the average stock of the predicted raw materials and the RMSE of the model.

In [12]:
def lr_analysis(data):
    # division of samples
    data_train, data_valid = train_test_split(data, test_size=0.25, random_state=12345) 

    # creating new variables
    features_train = data_train.drop(['id','product'], axis=1)
    target_train = data_train['product']
    features_valid = data_valid.drop(['id','product'], axis=1)
    target_valid = data_valid['product']
    
    # features scaling
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric]) 

    # training a model on a validation set
    lr_model = LinearRegression()
    lr_model.fit(features_train, target_train)
    predicted_valid = lr_model.predict(features_valid)
    predictions = pd.Series(predicted_valid)
    mse = mean_squared_error(target_valid, predictions)
    
    # average stock of raw materials and RMSE of the model
    rmse = mse ** 0.5
    avg_stock = sum(predictions) / len(predictions)
    
    # output
    print('Average stock of predicted raw materials:', avg_stock)
    print('RMSE of the model:', rmse)
    
    return (predictions, target_valid.reset_index(drop=True), rmse)    

Let's apply the written function for all datasets:

In [14]:
i = 1
for data in [df1, df2, df3]:
    print('')
    print(F"Region {i}:")
    lr_analysis(data)
    i+=1


Region 1:
Average stock of predicted raw materials: 92.59256778438008
RMSE of the model: 37.5794217150813

Region 2:
Average stock of predicted raw materials: 68.72854689544583
RMSE of the model: 0.8930992867756173

Region 3:
Average stock of predicted raw materials: 94.96504596800509
RMSE of the model: 40.02970873393434


As a result of the analysis, the following trends are visible:

- the largest average stock of predicted raw materials is in the wells of the third region, and the smallest - in the second region;
- the smallest RMSE = 0.9 and belongs to the predictions in the second region. At the same time, predictions in the first and third regions have large RMSE error rates, which indicates a not very good quality of model prediction.

In other words, our model did a more or less good job in predicting product volumes for the second region only. The predictions for regions 1 and 3 are far from the actual volumes.

## Preparation for calculating profit

**Saving all key values for calculations in separate variables:**

In [15]:
budget = 10000000000
oil_wells = 200
barrel_income = 450000

**Calculation of a sufficient volume of raw materials for a break-even development of a new well**

In [16]:
enough_volume = budget / oil_wells / barrel_income
print('Sufficient volume of raw materials for break-even development:', enough_volume)

Sufficient volume of raw materials for break-even development: 111.11111111111111


**Comparison of the received volume of raw materials with the average stock in each region**

In [17]:
# Calculation of the average stock of raw materials in each region:

def avg_volume(data):
    volume = sum(data['product']) / len(data['product'])
    print('Average stock of raw materials in the region = ', volume)

In [18]:
i = 1
for data in [df1, df2, df3]:
    print('')
    print(F"Region {i}:")
    avg_volume(data)
    i+=1
    print('')
print('Sufficient volume of raw materials for break-even development:', enough_volume)


Region 1:
Average stock of raw materials in the region =  92.49999999999976


Region 2:
Average stock of raw materials in the region =  68.82500000002561


Region 3:
Average stock of raw materials in the region =  95.00000000000041

Sufficient volume of raw materials for break-even development: 111.11111111111111


At this stage, the preparation of the profit calculation was able to reveal that the average value of raw materials reserves in all three regions is less than the sufficient volume of raw materials for breakeven development. The third region is closest to the value of the volume of raw materials for breakeven development. The first region comes in second place, while the second region comes in third place.

## Calculation of profit and risks

Let's write a function for calculating profit for selected wells and model predictions:

- select the wells with the maximum prediction values;
- summarize the target value of the volume of raw materials corresponding to these predictions;
- calculate the profit for the received volume of raw materials.

Firstly, let's write a function that will store the predicted and target oil volume values for each of the three datasets:

In [23]:
lr_model = LinearRegression()
def pred_tar(data):
    features = data.drop(['id', 'product'], axis = 1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.25, random_state = 0)                  
    lr_model.fit(features_train, target_train)
    predictions = lr_model.predict(features_valid)
    predictions = pd.Series(predictions)
    target = pd.Series(target_valid)
    return predictions, target

Let's save these values for each of the datasets for further profit calculation:

In [24]:
df1_pred, df1_target = pred_tar(df1)
df2_pred, df2_target = pred_tar(df2)
df3_pred, df3_target = pred_tar(df3)

Function for calculating profit in each of the three regions:

In [25]:
def revenue_calculation(predictions, target):
    predictions = pd.Series(predictions).reset_index(drop = True)
    target = pd.Series(target).reset_index(drop = True)
    best_predictions = predictions.sort_values(ascending = False)
    best_target = target[best_predictions.index][:200].sum()
    revenue = best_target.sum() * barrel_income
    profit = revenue - budget
    return profit

In [26]:
print('Profit in region 1: {0:.2f} bln. RUB.'.format(revenue_calculation(df1_pred, df1_target) / 1000000000))

Profit in region 1: 3.36 bln. RUB.


In [27]:
print('Profit in region 2: {0:.2f} bln. RUB.'.format(revenue_calculation(df2_pred, df2_target) / 1000000000))

Profit in region 2: 2.42 bln. RUB.


In [28]:
print('Profit in region 3: {0:.2f} bln. RUB.'.format(revenue_calculation(df3_pred, df3_target) / 1000000000))

Profit in region 3: 2.62 bln. RUB.


## Risks and revenue for each region:

**Applying Bootstrap Technique to Determine Profit Distribution**

In [29]:
state = np.random.RandomState(12345)

In [30]:
def bootstrap(predictions, target):
    target = target.reset_index(drop=True)
    values = []
    risk_values = 0
    
    for i in range(1000):
        target_subsample = target.sample(500, replace = True, random_state=state)
        predictions_subsample = predictions[target_subsample.index]
        revenue = revenue_calculation(predictions_subsample, target_subsample)
        values.append(revenue_calculation(predictions_subsample, target_subsample))
        
    
    values = pd.Series(values)
    confidence_interval = (values.quantile(0.025), values.quantile(0.975))
    avg_revenue = values.mean()
    risk = st.percentileofscore(values, 0)
    
    return (avg_revenue, confidence_interval, risk)

Application of this function for all three datasets:

In [31]:
i = 1
for predictions, target in zip([df1_pred, df2_pred, df3_pred],
                           [df1_target, df2_target, df3_target]):
    avg_revenue, confidence_interval, risk = bootstrap(predictions, target)
    print('')
    print(F"Region {i}:")
    print(F'Average profit in the region: {avg_revenue} RUB.')
    print(F'95% Confidence interval: {confidence_interval}')
    print(F'Risk of loss: {risk} %')
    i+=1
    print('')


Region 1:
Average profit in the region: 444486225.3681067 RUB.
95% Confidence interval: (-53170462.89962602, 949296653.1890134)
Risk of loss: 4.7 %


Region 2:
Average profit in the region: 491573277.28277457 RUB.
95% Confidence interval: (113115966.68988861, 870932298.8568313)
Risk of loss: 0.2 %


Region 3:
Average profit in the region: 388914311.3096654 RUB.
95% Confidence interval: (-147014174.38470182, 906065451.8287103)
Risk of loss: 7.7 %



**Summary:**

As can be seen from the calculations, region 2 is the most profitable for well development, since it has the lowest percentage of losses (only 1.4%, which turns out to be less than the predetermined level of 2.5%), as well as the largest average profit among all three regions. 

## Conclusion

As a result of the work done, the following steps were implemented:

- The data were examined and prepared for subsequent work;
- Using linear regression, a model was built that predicts the stock of raw materials in each of the three regions. The model showed the best RMSE when predicting product volumes in the second region. That is, among all three regions, our predictions were most closely approximated to the actual figures based on the data of the second region;
- A function was written to calculate the sufficient volume of raw materials in the well for its breakeven development. A value of 111 barrels was obtained, which is higher than the average stock of raw materials in each of the three regions;
- Using the bootstrap tool, the risks and rewards from each region were calculated, as well as the 95% confidence interval. These calculations made it possible to conclude that the only promising region is region number 2, since it brings the largest profit, and also has the lowest percentage of risk of losses (1.4%, which turns out to be less than the level of 2.5% set by the condition).